#  Analyse of Spatial Transcriptome data using VGP

This tutorial shows loading, preprocessing, VGP analyse of Spatial Transcriptome dataset.

## Import packages
Here, we’ll import scbean along with other popular packages.

In [1]:
from scbean.model import vgp
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

## Loading dataset
This tutorial uses spatial transcriptome data of human breast cancer (layer 2), which contains 14,789 genes measured on 251 spots. It can be downloaded from [Spatial Transcriptomics Research](https://www.spatialresearch.org/resources-published-datasets/doi-10-1126science-aaf2403/).

In [2]:
filepath = 'Users/wyr/data/Layer2_BC_count_matrix-1.tsv'
data = pd.read_csv(filepath, sep='\t')

In [3]:
data

,Unnamed: 0,GAPDH,USP4,MAPKAPK2,CPEB1,LANCL2,MCL1,TMEM109,TMEM189,ITPK1,...,TREML1,C12orf79,ZCCHC12,ZNF222,TRIM17,RNASEK,KSR2,PCDHGB4,ACOXL,CASQ2
0,17.907x4.967,1,1,1,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18.965x5.003,7,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18.954x5.995,5,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.846x5.993,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20.016x6.019,2,0,1,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,23.094x23.975,4,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247,24.981x23.964,4,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,21.874x24.852,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,23.096x24.93,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Each row represents a two-dimensional position and each column represents a gene.

## Preprocessing
Here, we separate location information and gene expression from the original data to meet the input of the model. In addition, we will filter out some genes and spots with low expression levels.

In [4]:
location = pd.DataFrame(index=data.index)
location['x'] = data['Unnamed: 0'].str.split('x').str.get(0).map(float)
location['y'] = data['Unnamed: 0'].str.split('x').str.get(1).map(float)
data.drop('Unnamed: 0', axis=1, inplace=True)
# Filter practically unobserved genes
data = data.T[data.sum(0) >= 10].T
# genes * location
data = data.T
location = location[data.sum(0) >= 10]
data = data.T[data.sum(0) >= 10].T
# model inputs
X = location.values
Y = data

## Identify SV genes via VGP
The result will be a DataFrame with p-value and adjusted p-value (q-value) for each gene.
We identified genes with q-value less than 0.05 as spatially variable genes.

In [5]:
if __name__ == '__main__':
    result = vgp.run(X, Y, mp.cpu_count())

  0%|                                                                                 | 1/9907 [00:00<23:16,  7.09it/s]

Results....................


100%|████████████████████████████████████████████████████████████████████████████| 9907/9907 [00:06<00:00, 1599.96it/s]


In [6]:
result

,gene,p_value,q_value
0,GAPDH,0.0,0.0
1,USP4,0.559464,0.559514
2,MAPKAPK2,0.000167,0.000167
3,CPEB1,0.611947,0.611988
4,LANCL2,0.892471,0.897269
...,...,...,...
9902,AC245100.1,0.263375,0.263539
9903,HSD17B6,0.571247,0.571286
9904,ARNTL,0.560723,0.560871
9905,PAQR8,0.000015,0.000015


According to the result, we screened the SV genes, that is, the genes with q-value less than 0.05.

In [7]:
result.sort_values(["q_value"], inplace=True)
len(result[result["q_value"]<0.05])

4129

In [8]:
result.head(10)

,gene,p_value,q_value
0,GAPDH,0.0,0.0
2857,IDH2,0.0,0.0
693,RPL30,0.0,0.0
2844,COL6A2,0.0,0.0
2833,METRN,0.0,0.0
2822,RAB11A,0.0,0.0
2816,DUSP14,0.0,0.0
2803,SET,0.0,0.0
706,C4A,0.0,0.0
707,RPS20,0.0,0.0


# Analysis of VGP result in R
We mapped the expression pattern of the SV genes based on R language and some R software packages.


**Import packages**

```R
library(ggplot2)
library(gridExtra)
library(grid)
```

**Load data**

```R
count <- read.csv("./data/Layer2_BC_expression.csv")
rownames(count) <- count[,1]
count <- as.data.frame(as.matrix(count[,2:ncol(count)]))
colnames(count) <- paste0("s",1:ncol(count))
info <- read.csv("./data/Layer2_BC_location.csv")
info <- as.data.frame(as.matrix(info[,2:ncol(info)]))
rownames(info) <- paste0("s",1:nrow(info))
```

**Define function**

```R
# anscombe variance stabilizing transformation: NB
var_stabilize <- function(x, sv = 1) {
  varx = apply(x, 1, var)
  meanx = apply(x, 1, mean)
  phi = coef(nls(varx ~ meanx + phi * meanx^2, start = list(phi = sv)))
  return(log(x + 1/(2 * phi)))
}

# Linear normalization
relative_func <- function(expres){
  maxd = max(expres)-min(expres)
  rexpr = (expres-min(expres))/maxd
  return(rexpr)
}

# Plot gene expression pattern
pattern_plot <- function(pltdat, igene, xy=T, main=F, title=NULL) {
  if (!xy) {
    xy <- matrix(as.numeric(do.call(rbind, strsplit(as.character(pltdat[,1]), split = "x"))), ncol = 2)
    rownames(xy) <- as.character(pltdat[, 1])
    colnames(xy) <- c("x", "y")
    pd <- cbind.data.frame(xy, pltdat[, 2:ncol(pltdat)])
  } else {
    pd <- pltdat
  }
  pal <- colorRampPalette(c("mediumseagreen", "lightyellow2", "deeppink"))
  gpt <- ggplot(pd, aes(x=x, y=y, color=pd[, igene + 2])) + geom_point(size=4) + 
    scale_color_gradientn(colours = pal(5)) + scale_x_discrete(expand=c(0, 1)) + 
    scale_y_discrete(expand=c(0, 1)) + coord_equal() + theme_bw()
  if (main) {
    if (is.null(title)) {
      title = colnames(pd)[igene + 2]
    }
    out = gpt + labs(title=title, x=NULL, y=NULL) + 
      theme(legend.position="none", plot.title=element_text(hjust=0.5, size=rel(1.5)))
  } else {
    out = gpt + labs(title=NULL, x=NULL, y=NULL) + theme(legend.position="none")
  }
  return(out)
}
```

**Plot gene expression pattern**

```R
count <- var_stabilize(count)
pltdat  <- cbind.data.frame(info[, 1:2], apply(count[c("GAPDH", "IDH2", "RPL30", "COL6A2", "METRN", "RAB11A", "DUSP14", "SET", "C4A", "RPS20"),], 1, relative_func))
plot <- lapply(1:10, function(x){pattern_plot(pltdat, x, xy = T, main = T)})
grid.arrange(grobs=plot[10:1], ncol=5)
```

<img src="https://github.com/jhu99/scbean/blob/vgp/docs/source/tutorials/Rplot_SV_genes.png?raw=true" width="70%">